In [1]:
from statsmodels.tsa.api import ExponentialSmoothing, \
SimpleExpSmoothing, Holt
import statsmodels.api as sm
y_hat_avg = test.copy()
fit2 = SimpleExpSmoothing(np.asarray(train['Count'])).fit(
smoothing_level=0.6,optimized=False)
y_hat_avg['SES'] = fit2.forecast(len(test))


In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import parameters
import load_data
path = parameters.G_DataPath
x_train, y_train, x_test = load_data.load_train_data(path)
df = pd.read_csv('data/data83497/jetrail.csv', nrows=11856)
train = df[0:10392]
test = df[10392:]
 
# Aggregating the dataset at daily level
df['Timestamp'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y %H:%M')  # 4位年用Y，2位年用y
df.index = df['Timestamp']
df = df.resample('D').mean() #按天采样，计算均值
 
train['Timestamp'] = pd.to_datetime(train['Datetime'], format='%d-%m-%Y %H:%M')
train.index = train['Timestamp']
train = train.resample('D').mean() #
 
test['Timestamp'] = pd.to_datetime(test['Datetime'], format='%d-%m-%Y %H:%M')
test.index = test['Timestamp']
test = test.resample('D').mean()
 
#Plotting data
train.Count.plot(figsize=(15,8), title= 'Daily Ridership', fontsize=14)
test.Count.plot(figsize=(15,8), title= 'Daily Ridership', fontsize=14)
plt.show()


In [ ]:
import statsmodels.api as sm
sm.tsa.seasonal_decompose(train['Count']).plot()
result = sm.tsa.stattools.adfuller(train['Count'])
plt.show()
from statsmodels.tsa.api import Holt
 
y_hat_avg = test.copy()
 
fit = Holt(np.asarray(train['Count'])).fit(smoothing_level=0.3, smoothing_slope=0.1)
y_hat_avg['Holt_linear'] = fit.forecast(len(test))
 
plt.figure(figsize=(16, 8))
plt.plot(train['Count'], label='Train')
plt.plot(test['Count'], label='Test')
plt.plot(y_hat_avg['Holt_linear'], label='Holt_linear')
plt.legend(loc='best')
plt.show()


In [ ]:
# 导⼊模块
from sklearn import model_selection
# 导⼊数据
import parameters
import load_data
path = parameters.G_DataPath
x_train, y_train, x_test = load_data.load_train_data(path)
df = pd.read_csv('data/data83497/jetrail.csv', nrows=11856)
# 根据train数据集建模
model = sm.formula.ols('Profit ~ RD_Spend+Administration+Marketing_Spend+C(State)', data= x_train).fit()
# print('模型的偏回归系数分别为：\n', model.params)
# 删除test数据集中的Profit变量，⽤剩下的⾃变量进⾏预测
test_X = test.drop(labels = 'Profit', axis = 1)

pred = model.predict(exog = test_X)
print('对⽐预测值和实际值的差异：\n',pd.DataFrame({'Prediction':pred,'Real':test.Profit}))

In [16]:
#定义线性回归模型
def test_LinearRegrssion():
    import parameters
    import load_data
    from sklearn import linear_model
    import pandas as pd 
    path = parameters.G_DataPath
    x_train, y_train, x_test = load_data.load_train_data(path)
    print(x_train.shape)
    regr = linear_model.LinearRegression()
    regr.fit(x_train,y_train)#训练数据
    print('Coefficients:%s,intercept %.2f' % (regr.coef_, regr.intercept_))#权重向量即为每个特征的相关系数
    y_test=regr.predict(x_test)
    data = pd.DataFrame(y_test)
    writer = pd.ExcelWriter('test.xlsx')
    data.to_excel(writer, 'page_1', float_format='%.5f')		# ‘page_1’是写入excel的sheet名
    writer.save()
    writer.close()
test_LinearRegrssion()

path -> ./data
x_train.shape -> (1596, 7)
y_train.shape -> (1596,)
x_test.shape  -> (798, 7)
(1596, 7)
Coefficients:[ 0.32542175  0.22815515 -0.11796135 -0.00827867 -0.63326764  0.43635374
 -0.11087308],intercept 0.31


D:\Programs\anaconda\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [18]:
def modifytraindata():
    # tn-2,tn-1,tn
    import preprocess
    import numpy as np
    a=np.loadtxt('./data/train_input.txt',dtype=np.float32)
    #data2删去前两行
    data1=np.delete(a, 0, axis = 0)
    data2=np.delete(data1, 0, axis = 0)
    #data3删去第一行和最后一行
    data3=np.delete(data1, len(data1)-1, axis = 0)
    #data4删去最后两行
    data4=np.delete(a, len(a)-1, axis = 0)
    data4=np.delete(data4, len(data4)-1, axis = 0)
    #按照data4、data3、data2排列
    data=np.concatenate((data4,data3),axis=1)
    data=np.concatenate((data,data2),axis=1)
    np.savetxt('./data/train_modify.txt',data)
    
def modifytestdata():
    import numpy as np
    a=np.loadtxt('./data/train_input.txt',dtype=np.float32)
    data1=a[1594:,:]
    print(data1.shape)
    b=np.loadtxt('./data/test_input.txt',dtype=np.float32)
    data0=np.concatenate((data1,b),axis=0)
    data1=np.delete(data0, len(data0)-1, axis = 0)
    data1=np.delete(data1, len(data1)-1, axis = 0)
    data2=np.delete(data0, len(data0)-1, axis = 0)
    data2=np.delete(data2, 0, axis = 0)
    data3=np.delete(data0, [0,1], axis = 0)
    data=np.concatenate((data1,data2,data3),axis=1)
    np.savetxt('./data/test_modify.txt',data)
def modifyoutput():
    a=np.loadtxt('./data/output.txt',dtype=np.float32)
    a=np.delete(a, [0,1], axis = 0)
    np.savetxt('./data/output_modify.txt',a)
def modifydata():
    modifytraindata()
    modifytestdata()
    modifyoutput()
modifydata()
import numpy as np
a=np.loadtxt('./data/train_modify.txt',dtype=np.float32)
print(a.shape)
a=np.loadtxt('./data/test_modify.txt',dtype=np.float32)
print(a.shape)

(2, 7)
(1594, 21)
(798, 21)
